# 변화 감지기

주기적으로 웹사이트에서 무엇이 변했는지 감지한다. 

In [6]:
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError

from bs4 import BeautifulSoup as bs

In [7]:
import difflib

In [20]:
target_sites = ['https://gall.dcinside.com/mgallery/board/lists/?id=momoreawqa', 'https://ddd.com']

In [21]:
def error_checker(site):
    try:
        html = urlopen(site)
    except HTTPError as e:
        print(site, "The server returned an HTTP error, ", e)
        return 1
    except URLError as e:
        print(site, "The server could not be found. ", e)
        return 1
    else: # Success
        return 0

In [26]:
filtered_target_sites = list(map(lambda x: x if error_checker(x)==0 else None, target_sites))
filtered_target_sites = list(filter(None, filtered_target_sites))
filtered_target_sites

https://ddd.com The server could not be found.  <urlopen error [Errno 11001] getaddrinfo failed>


['https://gall.dcinside.com/mgallery/board/lists/?id=momoreawqa']